In [ ]:
#' Remove baseline from amplification curves (BETA)
#'
#' Remove baseline from qPCR amplification curves
#' by subtracting median of initial cycles.
#'
#' BETA function version because:
#'
#' - assumes Roche Lightcycler format,
#' we should ideally replace "program_no == 2" by something more generic?
#'
#' - the rule-of thumb "baseline is median of initial 10 cycles"
#' has not been tested robustly
#'
#' @param plateamp data frame with plate amplification data, including variables
#'   well, cycle, fluor_raw (raw fluorescence value), and program_no. Assume
#'   program 2 for amplification curves from Roche Lightcycler format data.
#' @param maxcycle maximum cycle value to use for baseline, before
#'   amplification.
#'
#' @return platemap with additional columns per well:
#' \tabular{ll}{
#'   fluor_base      \tab baseline /background value  \cr
#'   fluor_signal    \tab normalized fluorescence signal, 
#'   i.e. fluor_raw - fluor_base
#'   }
#'
#' @export
#' @importFrom tidyr %>%
#' @importFrom rlang .data
#'
#' @examples
#' # create simple dataset of raw fluorescence
#' # with two samples over 15 cycles
#' raw_fluor_tibble <- tibble(sample_id = rep(c("S1", "S2"), each = 15),
#'                           target_id = "T1",
#'                           well_row = "A",
#'                           well_col = rep(c(1, 2), each = 15),
#'                           well = rep(c("A1", "A2"), each = 15),
#'                           cycle = rep(1:15,2),
#'                           fluor_raw = c(1:15, 6:20),
#'                           program_no = 2)
#'
#' # remove base fluorescence from dataset
#' raw_fluor_tibble %>%
#'     debaseline()
#'     
debaseline <- function(plateamp, maxcycle = 10) {
    assertthat::assert_that(
        assertthat::has_name(
            plateamp,
            c("well", "program_no", "cycle", "fluor_raw"))
    )
    assertthat::assert_that(
        !assertthat::has_name(plateamp,"fluor_base"),
        msg = "plateamp contains a variable called fluor_base, which breaks debaseline"
    )
    baseline <-
        plateamp %>%
        dplyr::group_by(.data$well) %>%
        dplyr::filter(.data$program_no == 2,
                      .data$cycle <= maxcycle) %>%
        dplyr::summarize(fluor_base = stats::median(.data$fluor_raw))
    plateamp %>%
        dplyr::left_join(baseline, by = "well") %>%
        dplyr::mutate(fluor_signal = .data$fluor_raw - .data$fluor_base)
}


#' Calculate dy/dx vector from vectors y and x
#'
#' Used in tidyqpcr to calculate dR/dT for a melt curve of fluorescence signal R
#' vs temperature T.
#'
#' @param x input variable, numeric vector, assumed to be temperature
#' @param y output variable, numeric vector of same length as x, assumed
#'   to be fluorescence signal.
#' @param method to use for smoothing:
#'
#'   "spline" default, uses smoothing spline stats::smooth.spline.
#'
#'   "diff" base::diff for lagged difference
#'
#' @param ... other arguments to pass to smoothing method.
#'
#' @return estimated first derivative of y with respect to x, numeric vector of
#'   same length as y.
#'
#' @family melt_curve_functions
#'
#' @export
#' 
#' @examples
#' # create simple curve
#' x = 1:5
#' y = x^2
#'
#' # calculate gradient of curve
#' #----- use case 1 : using splines
#' calculate_dydx(x, y)
#' 
#' # optional arguments are passed to smooth.splines function
#' calculate_dydx(x, y, spar = 0.5)
#' 
#' #----- use case 2 : using difference between adjacent points
#' calculate_dydx(x, y, method = "diff")
#'
calculate_dydx <- function(x, y, method = "spline", ...) {
    assertthat::assert_that(is.numeric(x))
    assertthat::assert_that(is.numeric(y))
    assertthat::assert_that(length(x) == length(y))
    if (method == "diff") {
       return(-c(diff(y) / diff(x), NA))
    } else if (method == "spline") {
        fit <- stats::smooth.spline(x = x, y = y, ...)
        return(-1 * stats::predict(object = fit, x = x, deriv = 1)$y)
    }
}


#' Calculate dR/dT of melt curves for of every well in a plate.
#'
#' dR/dT, the derivative of the melt curve (of fluorescence signal R vs
#' temperature T), has a maximum at the melting temperature Tm. A single peak in
#' this suggests a single-length PCR product is present in the well.
#' 
#' Note that this function does not group by plate, only by well.
#' The function will give strange results if you pass it data from 
#' more than one plate. Avoid this by analysing one plate at a time.
#'
#' @param platemelt data frame describing melt curves, including variables
#'   well, temperature, fluor_raw (raw fluorescence value).
#' @param method to use for smoothing:
#'
#'   "spline" default, uses smoothing spline stats::smooth.spline.
#'
#'   "diff" base::diff for lagged difference
#'
#' @param ... other arguments to pass to smoothing method.
#'
#' @return platemelt with additional column dRdT.
#'
#' @family melt_curve_functions
#'
#' @export
#' @importFrom tidyr %>%
#'
#' @examples
#' # create simple curve
#' # create simple dataset of raw fluorescence with two samples
#' temp_tibble <- tibble(sample_id = rep(c("S1", "S2"), each = 10),
#'                           target_id = "T1",
#'                           well_row = "A",
#'                           well_col = rep(c(1, 2), each = 10),
#'                           well = rep(c("A1", "A2"), each = 10),
#'                           temperature = rep(56:65,2),
#'                           fluor_raw = c(1:10, 6:15))
#'
#' # calculate drdt of all melt curves
#' #----- use case 1 : using splines
#' temp_tibble %>%
#'     calculate_drdt_plate()
#' 
#' # optional arguments are passed to smooth.splines function
#' temp_tibble %>%
#'     calculate_drdt_plate(spar = 0.5)
#' 
#' #----- use case 2 : using difference between adjacent points
#' temp_tibble %>%
#'     calculate_drdt_plate(method = "diff")
#'
calculate_drdt_plate <- function(platemelt, method = "spline", ...) {
    assertthat::assert_that(
        assertthat::has_name(
            platemelt,
            c("well", "temperature", "fluor_raw"))
    )
    if (assertthat::has_name(platemelt,"plate") ) {
        warning("platemelt has a plate column, but calculate_drdt_plate works only for single plates.")
    }
    platemelt %>%
        dplyr::arrange(.data$well, .data$temperature) %>%
        dplyr::group_by(.data$well) %>%
        dplyr::mutate(dRdT =
                          calculate_dydx(x = .data$temperature,
                                         y = .data$fluor_raw,
                                         method = method,
                                         ...)
        ) %>%
        dplyr::ungroup()
}

In [ ]:
#' Calculate a normalized value for a subset of reference ids
#'
#' This is used to calculate the normalized `cq` values for reference
#' `target_ids` (e.g. genes), to use in `delta_cq` calculation for each
#' `sample_id`.
#'
#' Also used to calculate the normalized `delta_cq` values for reference
#' `sample_ids`, to use in `deltadelta_cq` calculation for each `target_id`.
#'
#' @param value_df data frame containing relevant columns, those named in
#'   `value_name` and `id_name` parameters.
#' @param ref_ids values of reference ids, that are used to calculate
#'   normalized reference value.
#' @param value_name name of column containing values. This column should be
#'   numeric.
#' @param id_name name of column containing ids.
#' @param norm_function Function to use to calculate the value to normalize by.
#'   Default function is median, alternatively could use mean, geometric mean,
#'   etc.
#'
#' @export
#' @importFrom tidyr %>%
#' @importFrom stats median
#'
#' @examples
#' # create simple cq dataset with one sample, two targets  and 3 reps
#' cq_tibble <- tibble(sample_id = "S_1",
#'                      target_id = rep(c("T_1",
#'                                        "T_norm"), each = 3),
#'                      tech_rep = rep(1:3, 2),
#'                      well_row = rep(c("A",
#'                                       "B"), each = 3),
#'                      well_col = 1,
#'                      well = paste0(well_row, well_col),
#'                      cq = c(10, 10, 10,
#'                             12, 12, 11))
#'                      
#' # normalise cq to reference target_id called 'T_norm'
#' 
#' #----- use case 1: median reference target_id value
#' cq_tibble %>%
#'     calculate_normvalue(ref_ids = "T_norm",
#'                         value_name = "cq",
#'                         id_name = "target_id")
#' 
#' #----- use case 2: mean reference target_id value 
#' cq_tibble %>%
#'     calculate_normvalue(ref_ids = "T_norm",
#'                         value_name = "cq",
#'                         id_name = "target_id",
#'                         norm_function = mean)
#'
calculate_normvalue <- function(value_df,
                      ref_ids,
                      value_name = "value",
                      id_name = "id",
                      norm_function = median) {
    
    assertthat::assert_that(
        assertthat::has_name(value_df, 
                             c(value_name,id_name)))
    
    # make subset of value_df where gene is one or more ref_ids
    value_to_norm_by <- dplyr::filter(value_df,
                             .data[[id_name]] %in% ref_ids) %>%
        dplyr::pull({{value_name}}) %>%
        norm_function(na.rm = TRUE)
    
    # assign summary (median) value to value_df$value_to_norm_by
    dplyr::mutate(value_df, value_to_norm_by = value_to_norm_by)
}

#' Calculate delta cq (\eqn{\Delta Cq}) to normalize quantification cycle
#' (log2-fold) data within sample_id.
#'
#' This function implements relative quantification by the delta Cq method. For
#' each sample, the Cq values of all targets (e.g. genes, probes, primer sets)
#' are compared to one or more reference target ids specified in
#' `ref_target_ids`.
#'
#' @param cq_df a data frame containing columns `sample_id`, value_name (default
#'   `cq`) and tid_name (default `target_id`). Crucially, sample_id should be
#'   the same for different technical replicates measuring identical reactions
#'   in different wells of the plate, but differ for different biological and
#'   experimental replicates. See tidyqpcr vignettes for examples.
#' @param ref_target_ids names of targets to normalize by, i.e. reference
#'   genes, hydrolysis probes, or primer sets. This can be one reference target
#'   id, a selection of multiple target ids, or even all measured target ids. In
#'   the case of all of them, the delta Cq value would be calculated relative to
#'   the median (or other `norm_function`) of all measured targets.
#' @param norm_function Function to use to calculate the value to normalize by
#'   on given scale. Default is median, alternatively could use mean.
#'
#' @return data frame like cq_df with three additional columns:
#'
#'   \tabular{ll}{ ref_cq    \tab summary (median/mean) cq value for reference
#'   target ids \cr delta_cq  \tab normalized value, \eqn{\Delta Cq} \cr
#'   rel_abund \tab normalized ratio, \eqn{2^(-\Delta Cq)} }
#'
#' @export
#' @importFrom tidyr %>%
#' @importFrom stats median
#' @importFrom rlang .data
#' 
#' @examples
#' # create simple cq dataset with two samples, two targets  and 3 reps
#' cq_tibble <- tibble(sample_id = rep(c("S_1","S_1","S_1", "S_2", "S_2", "S_2"), 2),
#'                      target_id = rep(c("T_1",
#'                                        "T_norm"), each = 6),
#'                      tech_rep = rep(1:3, 4),
#'                      well_row = rep(c("A",
#'                                       "B"), each = 6),
#'                      well_col = rep(1:6, 2),
#'                      well = paste0(well_row, well_col),
#'                      cq = c(10, 10, 10, 12, 12, 11,
#'                              9,  9,  9,  9,  9,  9))
#'                      
#' # calculate deltacq using reference target_id called 'T_norm'
#' 
#' #----- use case 1: median reference target_id value
#' cq_tibble %>%
#'     calculate_deltacq_bysampleid(ref_target_ids = "T_norm")
#' 
#' #----- use case 2: mean reference target_id value 
#' cq_tibble %>%
#'     calculate_deltacq_bysampleid(ref_target_ids = "T_norm",
#'                                  norm_function = mean)
#'
calculate_deltacq_bysampleid <- function(cq_df,
                                         ref_target_ids,
                                         norm_function = median) {
    
    assertthat::assert_that(
        assertthat::has_name(cq_df, 
                             c("target_id", "sample_id","cq")))
    
    cq_df %>%
        dplyr::group_by(.data$sample_id) %>%
        dplyr::do(calculate_normvalue(.data,
                                   ref_ids = ref_target_ids,
                                   value_name = "cq",
                                   id_name = "target_id",
                                   norm_function = norm_function)) %>%
        dplyr::rename(ref_cq = .data$value_to_norm_by) %>%
        dplyr::ungroup() %>%
        dplyr::mutate(
               delta_cq    = .data$cq - .data$ref_cq,
               rel_abund   = 2^-.data$delta_cq)
}


#' Calculate delta delta cq (\eqn{\Delta \Delta Cq}) to globally normalize
#' quantification cycle (log2-fold) data across sample_id.
#'
#' By default, \eqn{\Delta \Delta Cq} is positive if a target is more highly
#' detected in the relevant sample, compared to reference samples. This can be
#' flipped by setting the parameter `ddcq_positive` to `FALSE`. In either case,
#' The fold change, \eqn{2^{\Delta \Delta Cq}}, is also reported.
#'
#' This function does a global normalization, where all samples are compared to
#' one or more reference samples specified in `ref_sample_ids`. There are other
#' experimental designs that require comparing samples in pairs or small groups,
#' e.g. a time course comparing `delta_cq` values against a reference strain at
#' each time point. For those situations, instead we recommend adapting code
#' from this function, changing the grouping variables used in to
#' `dplyr::group_by` to draw the contrasts appropriate for the experiment.
#'
#' @param deltacq_df a data frame containing columns `sample_id`, value_name
#'   (default `delta_cq`) and tid_name (default `target_id`). Crucially,
#'   sample_id should be the same for different technical replicates measuring
#'   identical reactions in different wells of the plate, but differ for
#'   different biological and experimental replicates.
#'
#'   Usually this will be a data frame that was output by
#'   `calculate_deltacq_bysampleid`.
#'
#' @param ref_sample_ids reference sample_ids to normalize by
#' @param norm_function Function to use to calculate the value to normalize by
#'   on given scale. Default is median, alternatively could use mean.
#' @param ddcq_positive (default TRUE) output \eqn{\Delta \Delta Cq} as positive
#'   if a target is more highly detected in the relevant sample, compared to
#'   reference samples.
#'
#' @return data frame like cq_df with three additional columns:
#'
#'   \tabular{ll}{ ref_delta_cq  \tab summary (median/mean) \eqn{\Delta Cq}
#'   value for target_id in reference sample ids \cr deltadelta_cq \tab the
#'   normalized value, \eqn{\Delta \Delta Cq} \cr fold_change   \tab the
#'   normalized fold-change ratio, \eqn{2^(-\Delta \Delta Cq)} }
#'
#' @export
#' @importFrom tidyr %>%
#' @importFrom stats median
#'
#' @examples
#' # create simple deltacq dataset with two samples, two targets and 3 reps
#' deltacq_tibble <- tibble(sample_id = rep(c("S_1","S_1","S_1", "S_norm", "S_norm", "S_norm"), 2),
#'                      target_id = rep(c("T_1",
#'                                        "T_2"), each = 6),
#'                      tech_rep = rep(1:3, 4),
#'                      well_row = rep(c("A",
#'                                       "B"), each = 6),
#'                      well_col = rep(1:6, 2),
#'                      well = paste0(well_row,well_col),
#'                      delta_cq = c(1, 1, 1, 3, 3, 2,
#'                                   4, 5, 4, 5, 5, 5))
#'                      
#' # calculate deltadeltacq using reference target_id called 'S_norm'
#' 
#' #----- use case 1: median reference sample_id value
#' deltacq_tibble %>%
#'     calculate_deltadeltacq_bytargetid(ref_sample_ids = "S_norm")
#' 
#' #----- use case 2: mean reference sample_id value 
#' deltacq_tibble %>%
#'     calculate_deltadeltacq_bytargetid(ref_sample_ids = "S_norm",
#'                                  norm_function = mean)
#'
calculate_deltadeltacq_bytargetid <- function(deltacq_df,
                                         ref_sample_ids,
                                         norm_function = median,
                                         ddcq_positive = TRUE) {
    
    assertthat::assert_that(
        assertthat::has_name(deltacq_df, 
                             c("target_id", "sample_id","delta_cq")))
    
    ddcq_factor <- (-1) ^ ddcq_positive
    
    deltacq_df %>%
        dplyr::group_by(.data$target_id) %>%
        dplyr::do(calculate_normvalue(.data,
                                   ref_ids = ref_sample_ids,
                                   value_name = "delta_cq",
                                   id_name = "sample_id",
                                   norm_function = norm_function)) %>%
        dplyr::rename(ref_delta_cq = .data$value_to_norm_by) %>%
        dplyr::ungroup() %>%
        dplyr::mutate(
               deltadelta_cq = ddcq_factor *
                   (.data$delta_cq - .data$ref_delta_cq),
               fold_change   = 2 ^ .data$deltadelta_cq)
}

In [ ]:
#' Calibrate primer sets / probes by calculating detection efficiency and
#' R squared
#'
#' Note efficiency is given in ratio, not per cent; multiply by 100 for that.
#'
#' @param cq_df_1 data frame with cq (quantification cycle) data,
#' 1 row per well.
#'
#' Must have columns cq, dilution.
#'
#' Assumes data are only for 1 probe/primer set/target_id, i.e. all values in
#' cq_df_1 are fit with the same slope.
#'
#' @param formula formula to use for log-log regression fit.
#'
#' Default value assumes multiple biological replicates,
#' cq ~ log2(dilution) + biol_rep.
#'
#' If only a single Biological Replicate, change to cq ~ log2(dilution).
#'
#' @return data frame with 1 single row, and columns:
#' efficiency, efficiency.sd, r.squared.
#'
#' @seealso calculate_efficiency_bytargetid
#'
#' @export
#' @importFrom tibble tibble
#' 
#' @examples
#' # create simple dilution dataset
#' dilution_tibble <- tibble(dilution = rep(c(1, 0.1, 0.001, 0.0001), 2),
#'                      cq = c(1, 3, 4, 6,
#'                             4, 5, 6, 7),
#'                      biol_rep = rep(c(1,2), each = 4),
#'                      target_id = "T1")
#'                      
#' # calculate primer efficiency
#' 
#' #----- use case 1: include difference across replicates in model
#' dilution_tibble %>%
#'     calculate_efficiency()
#' 
#' #----- use case 2: ignore difference across replicates
#' dilution_tibble %>%
#'     calculate_efficiency(formula = cq ~ log2(dilution))
#'
calculate_efficiency <- function(cq_df_1, formula = cq ~ log2(dilution) + biol_rep) {
    if (length(unique(cq_df_1$target_id)) > 1) {
            warning("multiple target_ids, did you mean calculate_efficiency_bytargetid?")
    }
    slopefit <- stats::lm(formula = formula, data = cq_df_1)
    slopefitsummary <- summary(slopefit)
    tibble(efficiency = -slopefit$coefficients[2],
           efficiency.sd = slopefitsummary$coefficients[2, 2],
           r.squared = slopefitsummary$r.squared)
}

#' Calibrate multiple probes by calculating detection efficiency and R squared
#'
#' See calibration vignette for example of usage.
#'
#' Note efficiency is given in ratio, not per cent; multiply by 100 for that.
#'
#' @param cq_df a data frame with cq (quantification cycle) data, 1 row per well
#'
#' Must have columns prep_type, target_id, cq, dilution.
#' Only prep_type=="+RT" columns are used.
#'
#' @param formula formula to use for log-log regression fit.
#'
#' Default value assumes multiple biological replicates,
#' cq ~ log2(dilution) + biol_rep.
#'
#' If only a single Biological Replicate, change to cq ~ log2(dilution).
#' If multiple sample_ids, change to cq ~ log2(dilution) + sample_id.
#'
#' See ?formula for background and help.
#'
#' @param use_prep_types prep_type column values to use, default "+RT" for RT-qPCR.
#'
#' By default, this includes only reverse-transcribed values in the efficiency
#' estimation, so excludes negative controls such as no-template and no-RT.
#'
#' To skip this filtering step, set use_prep_types=NA.
#'
#'
#' @return data frame with columns:
#' target_id, efficiency, efficiency.sd, r.squared.
#'
#' @seealso calculate_efficiency
#'
#' @export
#' @importFrom tidyr %>%
#' @importFrom rlang .data
#'
#' @examples
#' 
#' # create simple dilution dataset for two target_ids with two biological reps
#' dilution_tibble <- tibble(target_id = rep(c("T_1",
#'                                             "T_2"), each = 8),
#'                           well_row = rep(c("A",
#'                                            "B"), each = 8),
#'                           well_col = rep(1:8, 2),
#'                           well = paste0(well_row, well_col),
#'                           dilution = rep(c(1, 0.1, 0.001, 0.0001), 4),
#'                           cq = c(1, 3, 4, 6, 1, 3, 5, 7,
#'                                  4, 5, 6, 7, 3, 7, 8, 9),
#'                           biol_rep = rep(c(1, 1, 1, 1, 2, 2, 2, 2), 2),
#'                           prep_type = "+RT")
#'                      
#' # calculate primer efficiency for multiple targets
#' 
#' #----- use case 1: include difference across replicates in model
#' dilution_tibble %>%
#'     calculate_efficiency_bytargetid()
#' 
#' #----- use case 2: ignore difference across replicates
#' dilution_tibble %>%
#'     calculate_efficiency_bytargetid(formula = cq ~ log2(dilution))
#'
calculate_efficiency_bytargetid <- function(cq_df,
                           formula = cq ~ log2(dilution) + biol_rep,
                           use_prep_types="+RT") {
    assertthat::assert_that(assertthat::has_name(cq_df, "target_id"))
    
    if (!is.na(use_prep_types)) {
        assertthat::assert_that(assertthat::has_name(cq_df, "prep_type"))
        cq_df <- dplyr::filter(cq_df, .data$prep_type %in% use_prep_types)
    }
    cq_df %>%
        dplyr::group_by(.data$target_id) %>%
        dplyr::do(calculate_efficiency(.data, formula = formula)) %>%
        dplyr::ungroup()
}

In [ ]:
#' Create a blank plate template as a tibble (with helper functions for common plate sizes)
#'
#' For more help, examples and explanations, see the plate setup vignette:
#' \code{vignette("platesetup_vignette", package = "tidyqpcr")}
#'
#' @param well_row Vector of Row labels, usually LETTERS
#' @param well_col Vector of Column labels, usually numbers
#' @return tibble (data frame) with columns well_row, well_col, well. This
#'   contains all pairwise combinations of well_row and well_col, as well as
#'   individual well names. Both well_row and well_col are coerced to factors
#'   (even if well_col is supplied as numbers), to ensure order is consistent.
#'
#'   However, well is a character vector as that is the default behaviour of
#'   "unite", and display order doesn't matter.
#'
#'   Default value describes a full 384-well plate.
#'
#' @examples
#' create_blank_plate(well_row=LETTERS[1:2],well_col=1:3)
#' 
#' create_blank_plate_96well()
#' 
#' create_blank_plate_1536well()
#' 
#' # create blank 96-well plate with empty edge wells
#' 
#' create_blank_plate(well_row=LETTERS[2:7], well_col=2:11)
#' 
#' # create blank 1536-well plate with empty edge wells
#' 
#' full_plate_row_names <- make_row_names_lc1536()
#' 
#' create_blank_plate(well_row=full_plate_row_names[2:31], well_col=2:47)
#'
#' @family plate creation functions
#'
#' @export
#' @importFrom tibble tibble as_tibble
#' @importFrom tidyr %>%
#' @importFrom forcats as_factor
#' @importFrom rlang .data
#'
create_blank_plate <- function(well_row = LETTERS[1:16], well_col = 1:24) {
    tidyr::crossing(well_row = as_factor(well_row),
                    well_col = as_factor(well_col)) %>%
        as_tibble() %>%
        tidyr::unite("well", .data$well_row, .data$well_col, 
                     sep = "", remove = FALSE)
}

#' @describeIn create_blank_plate create blank 96-well plate
#' @export
#'
create_blank_plate_96well <- function() {
    create_blank_plate(well_row = LETTERS[1:8], well_col = 1:12)
}

#' Generates row names for the Roche Lightcycler (tm) 1536-well plates
#' 
#' Creates a vector containing 36 row names according to the labelling system 
#' used by the Roche Lightcycler (tm)
#' 
#' @return Vector of row names: Aa,Ab,Ac,Ad,Ba,...,Hd.
#' 
#' @family plate creation functions
#' 
#' @examples
#' make_row_names_lc1536()
#' 
#' @export
#'
make_row_names_lc1536 <- function() {
    paste0(rep(LETTERS[1:8], each = 4), letters[1:4])
}

#' Generates row names for the Labcyte Echo 1536-well plates 
#' 
#' Creates a vector containing 36 row names according to the 
#' labelling system used by the Labcyte Echo
#' 
#' @return Vector of row names: A,B,...,Z,AA,AB,...,AF.
#' 
#' @family plate creation functions
#' 
#' @examples
#' make_row_names_echo1536()
#' 
#' @export
#'
make_row_names_echo1536 <- function() {
    c(LETTERS[1:26], paste0("A", LETTERS[1:6]))
}

#' @describeIn create_blank_plate create blank 1536-well plate
#' @export
#'
create_blank_plate_1536well <- function(
    well_row = make_row_names_lc1536(),
    well_col = 1:48) {
    create_blank_plate(well_row, well_col)
}

#' Create a 6-value, 24-column key for plates
#'
#' Create a 24-column key with 6 values repeated over 24 plate columns.
#' Each of the 6 values is repeated over 3x +RT Techreps and 1x -RT.
#' 
#' This helps to create plate layouts with standard designs.
#'
#' @param ... Vectors of length 6 describing well contents,
#' e.g. sample_id or target_id
#' @return tibble (data frame) with 24 rows, and columns
#' well_col, prep_type, tech_rep, and supplied values.
#'
#' @examples
#' create_colkey_6_in_24(sample_id=LETTERS[1:6])
#' @family plate creation functions
#'
#' @export
#' @importFrom tibble tibble as_tibble

#' @importFrom tidyr %>%
#'
create_colkey_6_in_24 <- function(...) {
    colkey <- tibble(well_col  = factor(1:24),
                     prep_type = as_factor(c(rep("+RT", 18), rep("-RT", 6))),
                     tech_rep  = as_factor(rep(c(1, 2, 3, 1), each = 6))
                     )
    if (!missing(...)) {
        pieces6 <- list(...) %>% as_tibble()
        assertthat::assert_that(nrow(pieces6) == 6, 
                                msg = "Some input data is not of length 6")
        pieces24 <- dplyr::bind_rows(pieces6, pieces6, pieces6, pieces6)
        colkey <- dplyr::bind_cols(colkey, pieces24)
    }
    return(colkey)
}

#' Create a 4-dilution column key for primer calibration
#'
#' Creates a 24-column key for primer calibration, with 2x biol_reps and 2x
#' tech_reps, and 5-fold dilution until 5^4 of +RT; then -RT (no reverse
#' transcriptase), NT (no template) negative controls. That is a total of 6
#' versions of each sample replicate.
#'
#' @param dilution Numeric vector of length 6 describing sample dilutions
#' @param dilution_nice Character vector of length 6 with nice labels for sample
#'   dilutions
#' @param prep_type Character vector of length 6 describing type of sample (+RT,
#'   -RT, NT)
#' @param biol_rep Character vector of length 6 describing biological replicates
#' @param tech_rep Character vector of length 6 describing technical replicates
#' @return tibble (data frame) with 24 rows, and columns well_col, dilution,
#'   dilution_nice, prep_type, biol_rep, tech_rep.
#' @examples
#' create_colkey_4diln_2ctrl_in_24()
#' @family plate creation functions
#'
#' @export
#' @importFrom tibble tibble
#' @importFrom forcats as_factor
#'
create_colkey_4diln_2ctrl_in_24 <- function(
                     dilution      = c(5 ^ (0:-3), 1, 1),
                     dilution_nice = c("1x", "5x", "25x", "125x", "-RT", "NT"),
                     prep_type     = c(rep("+RT", 4), "-RT", "NT"),
                     biol_rep      = rep(c("A", "B"), each = 12,
                                        length.out = 24),
                     tech_rep      = rep(1:2, each = 6,
                                        length.out = 24)
                     ) {
    tibble(well_col = factor(1:24),
           dilution = rep(dilution, 4),
           dilution_nice = rep(dilution_nice, 4),
           prep_type = as_factor(rep(prep_type, 4)),
           biol_rep = as_factor(biol_rep),
           tech_rep = as_factor(tech_rep)
    )
}

#' Create a 6-dilution column key for primer calibration
#'
#' Creates a 24-column key for primer calibration, with 1x biol_reps and 3x
#' tech_reps, and 5-fold dilution until 5^6 of +RT; then -RT (no reverse
#' transcriptase), NT (no template) negative controls. That is a total of 8
#' versions of each replicate.
#'
#' @param dilution Numeric vector of length 8 describing sample dilutions
#' @param dilution_nice Character vector of length 8 with nice labels for sample
#'   dilutions
#' @param prep_type Character vector of length 8 describing type of sample (+RT,
#'   -RT, NT)
#' @param tech_rep Character vector of length 8 describing technical replicates
#' @return tibble (data frame) with 24 rows, and variables well_col, dilution,
#'   dilution_nice, prep_type, biol_rep, tech_rep.
#' @examples
#' create_colkey_6diln_2ctrl_in_24()
#' @family plate creation functions
#'
#' @export
#' @importFrom tibble tibble
#' @importFrom forcats as_factor
#'
create_colkey_6diln_2ctrl_in_24 <- function(
                     dilution = c(5 ^ (0:-5), 1, 1),
                     dilution_nice = c("1x", "5x", "25x", "125x",
                                    "625x", "3125x", "-RT", "NT"),
                     prep_type=c(rep("+RT", 6), "-RT", "NT"),
                     tech_rep = rep(1:3, each = 8, length.out = 24)
                     ) {
    tibble(well_col = factor(1:24),
           dilution = rep(dilution, 3),
           dilution_nice = rep(dilution_nice, 3),
           prep_type = as_factor(rep(prep_type, 3)),
           tech_rep = as_factor(tech_rep))
}

#' Create a 4-value, 16-row key for plates
#'
#' Create a 16-row key with 4 values repeated over 16 plate rows. Each of the 4
#' values is repeated over 3x +RT Techreps and 1x -RT.
#' 
#' This helps to create plate layouts with standard designs.
#'
#' @param ... Vectors of length 4 describing well contents, e.g. sample_id or
#'   target_id
#' @return tibble (data frame) with 16 rows, and variables well_row, prep_type,
#'   tech_rep, and supplied values.
#' @examples
#' create_rowkey_4_in_16(sample_id=c("sheep","goat","cow","chicken"))
#' @family plate creation functions
#'
#' @export
#' @importFrom tibble tibble as_tibble
#' @importFrom forcats as_factor
#' @importFrom tidyr %>%
#'
create_rowkey_4_in_16 <- function(...) {
    rowkey <- tibble(well_row = factor(LETTERS[1:16]),
                     prep_type = as_factor(c(rep("+RT", 12), rep("-RT", 4))),
                     tech_rep = as_factor(rep(c(1, 2, 3, 1), each = 4))
    )
    if (!missing(...)) {
        pieces4 <- list(...) %>% as_tibble()
        assertthat::assert_that(nrow(pieces4) == 4, 
                                msg = "Some input data is not of length 4")
        pieces16 <- dplyr::bind_rows(pieces4, pieces4, pieces4, pieces4)
        rowkey <- dplyr::bind_cols(rowkey, pieces16)
    }
    return(rowkey)
}

#' Create a plain 8-value, 16-row key for plates
#'
#' Create a 16-row key with 8 values repeated over 16 plate rows. No other
#' information is included by default, hence "plain".
#' 
#' This helps to create plate layouts with standard designs.
#'
#' @param ... Vectors of length 8 describing well contents, e.g. sample or
#'   probe.
#' @return tibble (data frame) with 16 rows, and variables well_col, and
#'   supplied values.
#' @examples
#' create_rowkey_8_in_16_plain(sample_id=c("me","you","them","him",
#'                                    "her","dog","cat","monkey"))
#' @family plate creation functions
#'
#' @export
#' @importFrom tibble tibble
#' @importFrom tidyr %>%
#'
create_rowkey_8_in_16_plain <- function(...) {
    rowkey <- tibble(well_row = factor(LETTERS[1:16]))
    if (!missing(...)) {
        pieces8 <- list(...) %>% as_tibble()
        assertthat::assert_that(nrow(pieces8) == 8, 
                                msg = "Some input data is not of length 8")
        pieces16 <- dplyr::bind_rows(pieces8, pieces8)
        rowkey <- dplyr::bind_cols(rowkey, pieces16)
    }
    return(rowkey)
}

#' Label a plate with sample and probe information
#'
#' For more help, examples and explanations, see the plate setup vignette:
#' \code{vignette("platesetup_vignette", package = "tidyqpcr")}
#' 
#' For worked examples of tidyqpcr analysis with 384-well plates, see:
#' \code{vignette("calibration_vignette", package = "tidyqpcr")}
#'
#' @param plate tibble (data frame) with variables well_row, well_col, well.
#'   This would usually be produced by create_blank_plate(). It is possible to
#'   include other information in additional variables.
#' @param rowkey tibble (data frame) describing plate rows, with variables
#'   well_row and others.
#' @param colkey tibble (data frame) describing plate columns, with variables
#'   well_col and others.
#' @param coercefactors if TRUE, coerce well_row in rowkey and well_col in
#'   colkey to factors
#'
#' @return tibble (data frame) with variables well_row, well_col, well, and
#'   others.
#'
#'   This tibble contains all combinations of well_row and well_col found in the
#'   input plate, and all information supplied in rowkey and colkey distributed
#'   across every well of the plate. Return plate is ordered by row well_row
#'   then column well_col.
#'
#'   Note this ordering may cause a problem if well_col is supplied as a
#'   character (1,10,11,...), instead of a factor or integer (1,2,3,...). For
#'   this reason, the function by default converts well_row in `rowkey`, and
#'   well_col in `colkey`, to factors, taking factor levels from `plate`, and
#'   messages the user.
#'   
#'   If `plate$well_col` or `plate$well_row` are not factors and coercefactors = TRUE 
#'   label_plate_rowcol will automatically convert them to factors, but will output a 
#'   warning telling users this may lead to unexpected behaviour. 
#'
#'   Other tidyqpcr functions require plate plans to contain variables
#'   sample_id, target_id, and prep_type, so `label_plate_rowcol` will message
#'   if any of these are missing. This is a message, not an error, because these
#'   variables can be added by users later.
#'
#' @examples
#' label_plate_rowcol(plate = create_blank_plate()) # returns blank plate
#' 
#' # label blank 96-well plate with empty edge wells
#' 
#' label_plate_rowcol(plate = create_blank_plate(well_row = LETTERS[2:7], 
#'                                               well_col = 2:11))
#' 
#' # label 96-well plate with sample id in rows
#' 
#' label_plate_rowcol(plate = create_blank_plate(well_row = LETTERS[1:8],
#'                                               well_col = 1:12),
#'                    rowkey = tibble(well_row = LETTERS[1:8],
#'                                    sample_id = paste0("S_",1:8)))
#' 
#' # label fraction of 96-well plate with target id in columns
#' 
#' label_plate_rowcol(plate = create_blank_plate(well_row = LETTERS[1:8],
#'                                               well_col = 1:4),
#'                    colkey = tibble(well_col = 1:4,
#'                                    target_id = paste0("T_",1:4)))
#' 
#' @family plate creation functions
#'
#' @export
#'
#' @importFrom rlang .data
#' @importFrom forcats as_factor
#'
label_plate_rowcol <- function(plate,
                               rowkey = NULL,
                               colkey = NULL,
                               coercefactors = TRUE) {
    assertthat::assert_that(
        assertthat::has_name(plate, 
                             c("well_row","well_col")))
    
    if (!is.factor(plate$well_col) & coercefactors){
        warning("plate$well_col is not a factor. Automatically generating plate$well_col factor levels. May lead to incorrect plate plans.")
        plate <- plate %>%
            dplyr::mutate(well_col = as_factor(.data$well_col))
    }
    
    if (!is.factor(plate$well_row) & coercefactors){
        warning("plate$well_row is not a factor. Automatically generating plate$well_row factor levels. May lead to incorrect plate plans.")
        plate <- plate %>%
            dplyr::mutate(well_row = as_factor(.data$well_row))
    }
    
    if (!is.null(colkey)) {
        assertthat::assert_that(assertthat::has_name(colkey, "well_col"))
        # Note: should this if clause be a freestanding function?
        # coerce_column_to_factor(df, col, warn=FALSE)?
        if (!is.factor(colkey$well_col) & coercefactors) {
            message("coercing well_col to a factor with levels from plate$well_col")
            colkey <- dplyr::mutate(
                colkey,
                well_col = factor(.data$well_col,
                                  levels = levels(plate$well_col))
            )
        }
        plate <- dplyr::left_join(plate, colkey, by = "well_col")
    }
    if (!is.null(rowkey)) {
        assertthat::assert_that(assertthat::has_name(rowkey, "well_row"))
        if (!is.factor(rowkey$well_row) & coercefactors) {
            message("coercing well_row to a factor with levels from plate$well_row")
            rowkey <- dplyr::mutate(
                rowkey,
                well_row = factor(.data$well_row,
                                  levels = levels(plate$well_row))
            )
        }
        plate <- dplyr::left_join(plate, rowkey, by = "well_row")
    }
    # check that plate contains sample_id, target_id, prep_type
    if (! "sample_id" %in% names(plate)) {
        message("plate does not contain variable sample_id")
    }
    if (! "target_id" %in% names(plate)) {
        message("plate does not have variable target_id")
    }
    if (! "prep_type" %in% names(plate)) {
        message("plate does not have variable prep_type")
    }
    return(dplyr::arrange(plate, .data$well_row, .data$well_col))
}


#' Display an empty plate plan which can be populated with 
#' ggplot2 geom elements.
#'
#' @param plate tibble with variables well_col, well_row.
#'
#' @return ggplot object; major output is to plot it
#'
#' @examples
#' library(ggplot2)
#' 
#' # display empty plot of empty plate
#' display_plate(create_blank_plate_96well())
#' 
#' # display wells of empty plate filled by column
#' display_plate(create_blank_plate_96well()) + 
#'   geom_tile(aes(fill = well_col), colour = "black")
#' 
#' # display wells of empty 1536-well plate filled by row
#' display_plate(create_blank_plate_1536well()) + 
#'   geom_tile(aes(fill = well_row), colour = "black")
#' 
#' @family plate creation functions
#'
#' @export
#' @importFrom forcats as_factor
#' @importFrom rlang .data
#'
display_plate <- function(plate) {
    assertthat::assert_that(
        assertthat::has_name(plate, 
                             c("well_row","well_col")))
    
    rowlevels <- 
        dplyr::pull(plate, .data$well_row) %>%
        as_factor() %>%
        levels()

    ggplot2::ggplot(data = plate,
                    ggplot2::aes(x = as_factor(.data$well_col),
                        y = as_factor(.data$well_row))) +
        ggplot2::scale_x_discrete(expand = c(0, 0)) +
        ggplot2::scale_y_discrete(expand = c(0, 0),
                                  limits = rev(rowlevels)) +
        ggplot2::coord_equal() +
        ggplot2::theme_void() +
        ggplot2::theme(axis.text = ggplot2::element_text(angle = 0),
                       panel.grid.major = ggplot2::element_blank(),
                       legend.position = "none",
                       plot.margin = grid::unit(rep(0.01, 4), "npc"),
                       panel.border = ggplot2::element_blank())
}

#' Display qPCR plate plan with sample_id, target_id, prep_type per well
#'
#' @param plate tibble with variables well_col, well_row, sample_id, target_id,
#'   prep_type. Output from label_plate_rowcol.
#'
#' @return ggplot object; major output is to plot it
#'
#' @examples 
#' 
#' 
#' # create basic 6-well plate
#' basic_plate <- 
#'     label_plate_rowcol(plate = create_blank_plate(well_row = LETTERS[1:2],
#'                                                   well_col = 1:3),
#'                        rowkey = tibble(well_row = factor(LETTERS[1:2]),
#'                                        target_id = c("T_A","T_B")),
#'                        colkey = tibble(well_col = factor(1:3),
#'                                        sample_id = c("S_1","S_2", "S_3"),
#'                                        prep_type = "+RT"))
#' 
#' # display basic plate
#' display_plate_qpcr(basic_plate)
#' 
#' # create full 384 well plate
#' full_plate <- label_plate_rowcol(create_blank_plate(), 
#'                                   create_rowkey_8_in_16_plain(target_id = c("T_1", "T_2",
#'                                                                             "T_3", "T_4", 
#'                                                                             "T_5", "T_6",
#'                                                                             "T_7", "T_8")), 
#'                                   create_colkey_6diln_2ctrl_in_24() %>% 
#'                                       dplyr::mutate(sample_id = paste0(dilution_nice,
#'                                                                        "_",
#'                                                                        tech_rep)))
#' 
#' # display full plate
#' display_plate_qpcr(full_plate)
#' 
#' @family plate creation functions
#'
#' @export
#' @importFrom rlang .data
#'
display_plate_qpcr <- function(plate) {
    assertthat::assert_that(
        assertthat::has_name(plate, 
                             c("target_id",
                               "sample_id",
                               "prep_type")))
    
    display_plate(plate) +
        ggplot2::geom_tile(ggplot2::aes(fill = .data$target_id), 
                           alpha = 0.3) +
        ggplot2::geom_text(ggplot2::aes(label = 
                                            paste(.data$target_id,
                                                  .data$sample_id,
                                                  .data$prep_type,
                                                  sep = "\n")),
                           size = 2.5, lineheight = 1)
}

#' Display the value of each well across the plate. 
#' 
#' Plots the plate with each well coloured by its value. Example values are Cq, Delta Cq or Delta Delta Cq.
#'
#' For a specific example see the calibration vignette:
#' \code{vignette("calibration_vignette", package = "tidyqpcr")}
#'
#' @param plate tibble with variables well_col, well_row, and the variable to be plotted.
#' 
#' @param value character vector selecting the variable in plate to plot as the well value
#'
#' @return ggplot object; major output is to plot it
#'
#' @examples 
#' library(dplyr)
#' library(ggplot2)
#' 
#' # create 96 well plate with random values
#' plate_randomcq <- create_blank_plate_96well() %>%
#'     mutate(cq = runif(96) * 10,
#'            deltacq = runif(96) * 2)
#' 
#' 
#' # display well Cq value across plate
#' display_plate_value(plate_randomcq)
#' 
#' # display well Delta Cq value across plate with red colour pallette
#' display_plate_value(plate_randomcq, value = "deltacq") +   # uses ggplot syntax
#'     scale_fill_gradient(high = "#FF0000") 
#'           
#' 
#' @family plate creation functions
#'
#' @export
#' @importFrom forcats as_factor
#' @importFrom rlang .data
#'
display_plate_value <- function(plate, value = "cq") {
    # check value exists in given plate
    assertthat::assert_that(value %in% names(plate), 
                            msg = paste0(value, " is not the name of a variable in the given plate"))
    
    # check each well has one value only
    unique_well_value <- plate %>%
        dplyr::group_by(.data$well) %>%
        dplyr::summarise(num_well = dplyr::n()) %>%
        dplyr::mutate(not_equal_one = .data$num_well != 1)
    
    assertthat::assert_that(sum(unique_well_value$not_equal_one) == 0, 
                            msg = paste0("Wells do not have unique ", value, " value."))
    
    rowlevels <- 
        dplyr::pull(plate, .data$well_row) %>%
        as_factor() %>%
        levels()
    
    display_plate(plate = plate) +
        ggplot2::geom_tile(ggplot2::aes(fill = .data[[value]])) +
        ggplot2::theme(plot.title = ggplot2::element_text(hjust = 0.5),
                       panel.grid.major = ggplot2::element_blank()) +
        ggplot2::labs(title = paste0({{value}}, " values for each well across the plate"))
}

In [ ]:
#' Reads raw text-format fluorescence data in 1 colour from Roche Lightcyclers
#' 
#' This is the data from "export in text format" from the Lightcycler software.
#' The other data format, .ixo, can be converted to .txt format by the
#' Lightcycler software.
#'
#' This function is a thin wrapper around readr::read_tsv.
#'
#' @param filename file name
#' @param skip number of lines to skip, defaults to 2
#' @param col_names names to give to columns
#' @param col_types data types of columns
#' @param ... other arguments to pass to read_tsv, if needed
#'
#' @return tibble containing raw data, with default column names:
#'
#'   well: the well of the plate, e.g. A1
#'
#'   sample_info: this is the "Sample" field entered in lightcycler software,
#'   defaults to "Sample X"
#'
#'   program_no: the number of the cycler program, for 2-step PCR defaults to 1
#'   = melt, 2 = amplify, 3 = melt.
#'
#'   segment_no: the number of the segment of the cycler program, e.g.
#'   hold/raise/lower temperature
#'
#'   cycle: the cycle number, for programs with repeated cycles (i.e.
#'   amplification)
#'
#'   time: the time of fluorescence reading acquisition (in what units???)
#'
#'   temperature: the temperature of the block at fluorescence acquisition
#'
#'   fluor_raw: the raw fluorescence reading in "arbitrary units". For SYBR safe, this
#'   would be 483nm excitation, 533nm emission.
#'   
#' @export
#' @seealso read_lightcycler_1colour_cq
#'
#' @examples read_lightcycler_1colour_raw(system.file("extdata/Edward_qPCR_Nrd1_calibration_2019-02-02.txt.gz", 
#'                                                  package = "tidyqpcr"))
#'
read_lightcycler_1colour_raw <- function(
    filename,
    skip = 2,
    col_names = c(
        "well", "sample_info", "program_no", "segment_no",
        "cycle", "time", "temperature", "fluor_raw"
    ), 
    col_types = "ccffinnn",
    ...) {
    readr::read_tsv(file = filename,
                    skip = skip,
                    col_names = col_names,
                    col_types = col_types,
                    ...)
}

#' Reads quantification cycle (cq) data in 1 colour from Roche Lightcyclers
#'
#' This is the data from "export in text format" from the analysis tab in the
#' Lightcycler software. That software calls cq, "Cp".
#'
#' This function is a thin wrapper around readr::read_tsv.
#'
#' @param filename file name
#' @param skip number of lines to skip, defaults to 2
#' @param col_names names to give to columns
#' @param col_types data types of columns
#' @param ... other arguments to pass to read_tsv, if needed
#'
#' @return tibble containing cq data
#' @export
#' @seealso read_lightcycler_1colour_raw
#'
#' @examples read_lightcycler_1colour_cq(system.file("extdata/Edward_qPCR_Nrd1_calibration_2019-02-02_Cq.txt.gz", 
#'                                                  package = "tidyqpcr"))
#'
read_lightcycler_1colour_cq <- function(
    filename,
    skip = 2,
    col_names = c(
        "include", "color", "well", "sample_info",
        "cq", "concentration", "standard", "status"
    ), 
    col_types = "liccddil",
    ...) {
    readr::read_tsv(file = filename,
                    skip = 2,
                    col_names=col_names,
                    col_types=col_types,
                    ...)
}